In [18]:
!streamlit run gizaAgentTest-INS-LIN.py

In [3]:
import argparse
import logging
import os
import pprint
from logging import getLogger

In [4]:
import numpy as np

In [5]:
import sys
print(sys.executable)

c:\Users\sathy\AppData\Local\Programs\Python\Python311\python.exe


In [6]:
from dotenv import find_dotenv, load_dotenv

In [7]:
from giza.agents import AgentResult, GizaAgent


c:\Users\sathy\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from addresses import ADDRESSES

In [9]:
# from lp_tools import get_tick_range
# from uni_helpers import (approve_token, check_allowance, close_position,
#                          get_all_user_positions, get_mint_params)

In [10]:
load_dotenv(find_dotenv())

True

In [11]:
dev_passphrase = os.environ.get("DEV_PASSPHRASE")
sepolia_rpc_url = os.environ.get("SEPOLIA_RPC_URL")

print(dev_passphrase)
print(sepolia_rpc_url)

dev_passphrase = "catestHack2024"
sepolia_rpc_url = os.environ.get("SEPOLIA_RPC_URL")

catestHack2024
https://eth-sepolia.g.alchemy.com/v2/alcht_qTqyinch9eBi0vF09MR064gABbQbLI


In [12]:
logging.basicConfig(level=logging.INFO)

In [13]:
def create_agent(
    model_id: int, version_id: int, chain: str, contracts: dict, account: str
):
    """
    Create a Giza agent for the regression model
    """
    agent = GizaAgent(
        contracts=contracts,
        id=model_id,
        version_id=version_id,
        chain=chain,
        account=account,
    )
    return agent

In [14]:
def predict(agent: GizaAgent, X: np.ndarray):
    """
    Predict the next day volatility.

    Args:
        X (np.ndarray): Input to the model.

    Returns:
        int: Predicted value.
    """
    prediction = agent.predict(input_feed={"val": X}, verifiable=True, job_size="XL")
    return prediction

In [15]:
def get_pred_val(prediction: AgentResult):
    """
    Get the value from the prediction.

    Args:
        prediction (dict): Prediction from the model.

    Returns:
        int: Predicted value.
    """
    # This will block the executon until the prediction has generated the proof
    # and the proof has been verified
    return prediction.value[0][0]

In [16]:
def eval_quote(
    #agent_id: int,
    pred_model_id:int,
    pred_version_id:int,
    policyId:int,
    quote: float,
    bmi:float,
    #chainURL:str,
    #chain=f"ethereum:sepolia:{SEPOLIA_RPC_URL}",
    account="chainaimlabs3003",
    chain=f"ethereum:sepolia:{sepolia_rpc_url}",
):
    
    ## Change the INS-ENROLLMENT-AGENT_PASSPHRASE to be {AGENT-NAME}_PASSPHRASE
    # os.environ["INS_ENROLLMENT-AGENT_PASSPHRASE"] = os.environ.get("DEV_PASSPHRASE")

    # Create logger

    logger = getLogger("agent_logger");

    #chain=f"ethereum:sepolia:{SEPOLIA_RPC_URL}",
    #networks.parse_network_choice(f"ethereum:sepolia:{SEPOLIA_RPC_URL}").__enter__()

    chain_id = 11155111;
    #chain_id = chain.chain_id;
    print (chain_id);

    # Load the addresses

    #insuranceEnrollmentContractAddress = ADDRESSES["INSE"][chain_id]
    insuranceEnrollmentContractAddress = "0x96d3441592F87CE5d89C8220DCbef3cFd06a9976";
    #insuranceEnrollmentContractAddress = "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14";

    # Load the data, this can be changed to retrieve live data
    #file_path = "data/data_array.npy"
    #X = np.load(file_path)
     
    input = np.array([[bmi]]).astype(np.float32)
    print(input);

    # Fill this contracts dictionary with the contract addresses that our agent will interact with
    contracts = {
        "insuranceEnrollmentContract": insuranceEnrollmentContractAddress,
        "WETH":  "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",
    }
    # Create the agent
    agent = create_agent(
        model_id=pred_model_id,
        version_id=pred_version_id,
        chain=chain,
        contracts=contracts,
        account=account,
    )

    result = predict(agent, input)
    print(result);



#



    with agent.execute() as contracts:

      
        logger.info("Executing INSE contract with policyId  "  )
        #contracts.WETH.deposit(value = 1000);
        contracts.insuranceEnrollmentContract.buyPolicy(policyId,value =100);
        
        res = contracts.insuranceEnrollmentContract.insurerWallet();
        print(res);

        logger.info("Executing INSE contract");
    
    """
    Get the value from the prediction.

    Args:
        prediction (dict): Prediction from the model.

    Returns:
        int: 

    """
    # This will block the executon until the prediction has generated the proof
    # and the proof has been verified

    print(result)
      }
    

    return result

In [17]:
if __name__ == "__main__":
   
   pred_model_id = 719;
   pred_version_id = 3;
   policyId = 1;
   quote = 100;
   bmi = 31.1;

   eval_quote(pred_model_id,pred_version_id,policyId,quote,bmi);
   

11155111
[[31.1]]


INFO:giza.agents.model:Could not download model: 'GizaAgent' object has no attribute '_output_path'
INFO:giza.agents.model:Model already downloaded at: C:\Users\sathy\AppData\Local\Temp\719_3_linear_regression-ins ✅
INFO:giza.agents.model:Predicting
INFO:giza.agents.model:Serialized: [1 1] [881822195 false]


🚀 Starting deserialization process...
✅ Deserialization completed! 🎉


INFO:giza.agents.agent:Updating agent with chain ethereum:sepolia:https://eth-sepolia.g.alchemy.com/v2/alcht_qTqyinch9eBi0vF09MR064gABbQbLI
INFO:giza.agents.agent:Updating agent with latest contracts


AgentResult(input={'val': array([[31.1]], dtype=float32)}, request_id=82ce3eec5a11471996f13129b113e623, value=[[13455.53886414]])


...

INFO:ape:Connecting to existing Geth node at https://eth-sepolia.g.alchemy.com/[hidden].


INFO:agent_logger:Executing INSE contract with policyId  


INFO: Submitted https://sepolia.etherscan.io/tx/0x460ec2554bd97bc02071b54b1c75bee9b295f9a8c74bce52a9f2f445d15922fd


INFO:ape:Submitted https://sepolia.etherscan.io/tx/0x460ec2554bd97bc02071b54b1c75bee9b295f9a8c74bce52a9f2f445d15922fd
Confirmations (2/2): 100%|██████████| 2/2 [00:28<00:00, 14.19s/it]


ProviderError: 400 Client Error: Bad Request for url: https://eth-sepolia.g.alchemy.com/v2/alcht_qTqyinch9eBi0vF09MR064gABbQbLI